In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import holidays
from typing import Optional

In [2]:
file_path = './data/'
TICKER = '^GSPC'
TIME_FRAME = '1d'

In [3]:
# If true, the new 5 minute data is used
USE_NEW_DATA = False

In [4]:
# Get the 5 minute data
start_index, end_index = None, None

if USE_NEW_DATA:
    df_small = pd.read_csv('./data/es-5m.csv', header=None, sep=';')
    df_small.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df_small.Date = pd.to_datetime(df_small.Date + ' ' + df_small.Time, dayfirst=True)
    df_small.drop(columns=['Time'], inplace=True)
    start_index, end_index = 83, len(df_small)

else:
    df_small = pd.read_csv(f'./data/ESU24.CME_5m.csv')
    df_small.Date = pd.to_datetime(df_small.Date)
    df_small.rename(columns={'Last': 'Close'}, inplace=True)
    start_index, end_index = 107, -158

assert start_index is not None and end_index is not None
df_small = df_small.iloc[start_index:end_index, :] # Quick hack to ensure we start and end with full days
df_small.reset_index(drop=True, inplace=True)
df_small

,ESU24,Date,Open,High,Low,Close,Volume
0,ESU24,2010-01-04 00:00:00,1240.50,1240.75,1240.00,1240.25,916
1,ESU24,2010-01-04 00:05:00,1240.50,1242.50,1240.00,1241.75,4898
2,ESU24,2010-01-04 00:10:00,1241.75,1242.75,1241.75,1242.25,3808
3,ESU24,2010-01-04 00:15:00,1242.25,1243.00,1242.25,1242.75,2469
4,ESU24,2010-01-04 00:20:00,1242.75,1243.25,1241.50,1242.00,3527
...,...,...,...,...,...,...,...
1017682,ESU24,2024-08-07 23:35:00,5234.25,5235.75,5232.50,5234.75,1165
1017683,ESU24,2024-08-07 23:40:00,5235.00,5235.25,5231.25,5233.50,1155
1017684,ESU24,2024-08-07 23:45:00,5233.50,5234.00,5229.50,5231.25,1259
1017685,ESU24,2024-08-07 23:50:00,5231.50,5231.75,5225.75,5230.25,1075


In [5]:
# Convert the 5 minute data to daily data (so that it's on the same scale)
df_small.set_index('Date', inplace=True)
df = df_small.resample('D').agg({
    'Open': 'first',
    'Close': 'last',   
    'High': 'max',     
    'Low': 'min'       
})
df_small.reset_index(inplace=True)
df.reset_index(inplace=True)
df.Date = pd.to_datetime(df.Date)
df['Date'] = df['Date'].dt.normalize()
us_holidays = holidays.US()
df['Holiday'] = df['Date'].apply(lambda x: 1 if x.strftime('%Y-%m-%d') in us_holidays else 0)
df

,Date,Open,Close,High,Low,Holiday
0,2010-01-04,1240.50,1251.75,1253.25,1240.00,0
1,2010-01-05,1251.75,1253.50,1256.50,1248.50,0
2,2010-01-06,1253.50,1254.00,1259.00,1250.75,0
3,2010-01-07,1254.25,1262.00,1263.00,1250.50,0
4,2010-01-08,1262.00,1265.00,1265.25,1254.50,0
...,...,...,...,...,...,...
5325,2024-08-03,NaN,NaN,NaN,NaN,0
5326,2024-08-04,5330.00,5238.50,5345.50,5201.00,0
5327,2024-08-05,5238.75,5271.50,5305.50,5120.00,0
5328,2024-08-06,5271.50,5294.75,5342.00,5221.75,0


In [6]:
class Trade:
    def __init__(self, trade_type: str, open_price: float, sl: float, sg: float, n_lots: int) -> None:
        self.trade_type = trade_type
        self.open_price = open_price
        self.sl = sl
        self.sg = sg
        self.n_lots = n_lots

    def is_buy(self) -> bool:
        return self.trade_type == 'buy'

In [7]:
class SimulationResults:
    def __init__(self) -> None:
        self.points = 0
        self.n_wins = 0
        self.n_losses = 0
        self.win_streak = 0
        self.loss_streak = 0
        self.curr_win_streak = 0
        self.curr_loss_streak = 0
        self.n_buys = 0
        self.n_sells = 0
        self.n_ticks = 0
        self.dollars = 0

    def update(self, trade: Trade, trade_amount: float) -> None:
        self.n_buys += 1 if trade.is_buy() else 0
        self.n_sells += 1 if not trade.is_buy() else 0
        self.points += trade_amount
        n_ticks = trade_amount * 4
        dollars = n_ticks * 1.25 * trade.n_lots
        self.n_ticks += n_ticks
        self.dollars += dollars - 0.4
        self.n_wins += 1 if trade_amount > 0 else 0
        self.n_losses += 1 if trade_amount < 0 else 0
        self.curr_win_streak = (self.curr_win_streak + 1) if trade_amount > 0 else 0
        self.curr_loss_streak = (self.curr_loss_streak + 1) if trade_amount < 0 else 0
        self.win_streak = max(self.win_streak, self.curr_win_streak)
        self.loss_streak = max(self.loss_streak, self.curr_loss_streak)

    def __str__(self) -> str:
        return f'Points: {self.points}\nTicks won: {self.n_ticks}\nDollars won: {self.dollars}\nNum buys: {self.n_buys}\nNum sells: {self.n_sells}\nNum wins: {self.n_wins}\nNum losses: {self.n_losses}\nWin streak: {self.win_streak}\nLoss streak: {self.loss_streak}'

In [8]:
def run_simulation(num_points_diff: int, stop_gain_points: int, stop_loss_points: int, remove_holidays: bool, n_lots: int, can_buy: bool, can_sell: bool, reverse: bool) -> SimulationResults:
    place_trade, buy, sim_results = False, False, SimulationResults()
    points_diff = num_points_diff
    sg_points, sl_points = stop_gain_points, stop_loss_points
    i = 0

    def _iterate_through_trade(start_date: datetime, place_buy: bool) -> Optional[datetime]:
        curr_df_small = df_small[df_small['Date'] > start_date]
        curr_df_small.reset_index(drop=True, inplace=True)

        if len(curr_df_small) == 0:
            return None

        open_price = curr_df_small.loc[curr_df_small.index[0], 'Open']

        if place_buy:
            sl = open_price + 1 - sl_points
            sg = open_price + 1 + sg_points
            trade = Trade('buy', open_price, sl, sg, n_lots)

        else:
            sl = open_price - 1 + sl_points
            sg = open_price - 1 - sg_points
            trade = Trade('sell', open_price, sl, sg, n_lots)


        for j in range(len(curr_df_small)):
            curr_date, curr_low, curr_high = curr_df_small.loc[curr_df_small.index[j], ['Date', 'Low', 'High']]

            if trade.is_buy():
                if curr_low <= trade.sl:
                    trade_amount = trade.sl - trade.open_price
                    sim_results.update(trade, trade_amount)
                    return curr_date

                if curr_high >= trade.sg:
                    trade_amount = trade.sg - trade.open_price
                    sim_results.update(trade, trade_amount)
                    return curr_date

            else:
                if curr_high >= trade.sl:
                    trade_amount = trade.open_price - trade.sl
                    sim_results.update(trade, trade_amount)
                    return curr_date

                if curr_low <= trade.sg:
                    trade_amount = trade.open_price - trade.sg
                    sim_results.update(trade, trade_amount)
                    return curr_date
                
        return None

    while i < len(df_small):
        if not place_trade:
            # Current 5 minute values
            date, curr_close = df_small.loc[df_small.index[i], ['Date', 'Close']]

            # Current day values
            curr_day, curr_month, curr_year = date.day, date.month, date.year
            search_date = pd.Timestamp(day=curr_day, month=curr_month, year=curr_year)
            daily_row = df[df['Date'] == search_date]
            if daily_row.empty:
                i += 1
                continue
            curr_open, is_holiday = daily_row.loc[daily_row.index[0], ['Open', 'Holiday']]

            # Place a trade (if the conditions are met)
            not_a_holiday = not (remove_holidays and is_holiday)
            buy_condition = curr_close > curr_open if reverse else curr_close < curr_open
            sell_condition = curr_close < curr_open if reverse else curr_close > curr_open

            if can_buy and buy_condition and abs(curr_open - curr_close) >= points_diff and not_a_holiday:
                place_trade, buy = True, True

            elif can_sell and sell_condition and abs(curr_open - curr_close) >= points_diff and not_a_holiday:
                place_trade, buy = True, False

        if place_trade:
            trade_end_date = _iterate_through_trade(date, buy)
            place_trade, buy = False, False

            if trade_end_date is None:
                break

            else:
                i = int(df_small.loc[df_small['Date'] <= trade_end_date].index[-1] + 1)

        else:
            i += 1

    return sim_results

In [ ]:
num_points = [20, 30, 50, 100]
sg_points = [20, 50, 100]
sl_points = [20, 50, 100]
remove_holidays_vals = [False]
can_buy_vals = [True, False]
can_sell_vals = [True, False]
reverse_vals = [True, False]
all_combos = []

for n_points in num_points:
    for sg_point in sg_points:
        for sl_point in sl_points:
            for remove_holiday in remove_holidays_vals:
                for can_b in can_buy_vals:
                    for can_s in can_sell_vals:
                        for reverse in reverse_vals:
                            if not can_b and not can_s:
                                continue
                            all_combos.append((n_points, sg_point, sl_point, remove_holiday, can_b, can_s, reverse))

best_num_points = None
best_sg_points = None
best_sl_points = None
best_remove_holidays_vals = None
best_can_buy_vals = None
best_can_sell_vals = None
best_reverse_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished, n_runs = 0, len(all_combos)
best_rewards = []
best_results = None

print(f'N runs: {n_runs}')

for n_points, sg_point, sl_point, remove_holiday, can_b, can_s, reverse in all_combos:
    results = run_simulation(num_points_diff=n_points, stop_gain_points=sg_point, stop_loss_points=sl_point, remove_holidays=remove_holiday, n_lots=1, can_buy=can_b, can_sell=can_s, reverse=reverse)
    profit = results.dollars
    runs_finished += 1

    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
    if min_item is None or profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': profit, 'n_points': n_points, 'sg_points': sg_point, 'sl_points': sl_point, 'remove_holiday': remove_holiday, 'can_buy': can_b, 'can_sell': can_s, 'reverse': reverse})

    if profit > best_reward:
        best_num_points = n_points
        best_sg_points = sg_point
        best_sl_points = sl_point
        best_remove_holidays_vals = remove_holiday
        best_can_buy_vals = can_b
        best_can_sell_vals = can_s
        best_reverse_val = reverse
        best_results = results
        best_reward = profit


    print('Best profit so far: ' + str(best_reward) + '\n')

N runs: 216


In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best distance points: ' + str(best_num_points))
print('Best sg points: ' + str(best_sg_points))
print('Best sl points: ' + str(best_sl_points))
print('Best remove holiday val: ' + str(best_remove_holidays_vals))
print('Best can buy val: ' + str(best_can_buy_vals))
print('Best can sell val: ' + str(best_can_sell_vals))
print('Best reverse val: ' + str(best_reverse_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 13607.200000000039
Best distance points: 20
Best sg points: 100
Best sl points: 100
Best remove holiday val: False
Best can buy val: True
Best can sell val: False
-----------------------
Top results:
{'reward': 7012.600000000025, 'n_points': 20, 'sg_points': 20, 'sl_points': 50, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 6184.400000000014, 'n_points': 20, 'sg_points': 20, 'sl_points': 100, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 8953.60000000008, 'n_points': 20, 'sg_points': 50, 'sl_points': 50, 'remove_holiday': False, 'can_buy': True, 'can_sell': True}
{'reward': 5665.400000000007, 'n_points': 20, 'sg_points': 50, 'sl_points': 50, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 7041.200000000028, 'n_points': 20, 'sg_points': 50, 'sl_points': 100, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 5099.4000000000215, 'n_points':

In [ ]:
print('Best results:')
print(best_results)

Best results:
Points: 2732.0
Ticks won: 10928.0
Dollars won: 13607.200000000039
Num buys: 132
Num sells: 0
Num wins: 79
Num losses: 53
Win streak: 8
Loss streak: 7
